In [ ]:
import sys
import os
import glob
import cv2
import numpy as np
import mediapipe as mp
from pose_format import Pose
from pose_format.utils.holistic import load_holistic
from pose_format.utils.openpose import load_openpose_directory
from pose_format.pose_visualizer import PoseVisualizer

mp_holistic = mp.solutions.holistic
FACEMESH_CONTOURS_POINTS = [str(p) for p in sorted(set([p for p_tup in list(mp_holistic.FACEMESH_CONTOURS) for p in p_tup]))]

def load_video_frames(cap: cv2.VideoCapture):
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        yield cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    cap.release()

def pose_estimate(video_path, output_path, lib='mediapipe', reduce=False):
    # Load video frames
    print('Load video ...')
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise FileNotFoundError(f"Cannot open video: {video_path}")

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = (cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Video metadata: fps={fps}, width={width}, height={height}, frames={total_frames}")

    frames = load_video_frames(cap)
    video_metadata = dict(fps=fps, width=width, height=height)

    # Perform pose estimation
    print('Estimating pose ...')

    if lib == 'mediapipe':
        pose = load_holistic(frame_gen(), fps=fps, width=width, height=height, progress=True,
                         additional_holistic_config={'model_complexity': 2, 'refine_face_landmarks': True})
    pose = pose.get_components(["POSE_LANDMARKS", "FACE_LANDMARKS", "LEFT_HAND_LANDMARKS", "RIGHT_HAND_LANDMARKS"])

    if reduce:
        pose = pose.get_components(["POSE_LANDMARKS", "FACE_LANDMARKS", "LEFT_HAND_LANDMARKS", "RIGHT_HAND_LANDMARKS"],
                                   {"FACE_LANDMARKS": FACEMESH_CONTOURS_POINTS})
    elif lib == 'openpose':
        pose = load_openpose_directory(video_path.replace('.mp4', '.openpose'), fps=fps, width=width, height=height)

    print('Points:', pose.body.data.shape)

    # Write output
    print('Writing pose file ...', output_path)
    with open(output_path, "wb") as f:
        pose.write(f)

def pose_visualize(video_path, pose_path, overlay=False):
    print('Reading pose file ...')
    with open(pose_path, "rb") as f:
        buffer = f.read()
        pose = Pose.read(buffer)

    v = PoseVisualizer(pose, thickness=2)

    if overlay:
        out_file = f"{pose_path}.overlay.mp4"
        print("Saving overlay visualization:", out_file)
        v.save_video(out_file, v.draw_on_video(video_path))
    else:
        out_file = f"{pose_path}.mp4"
        print("Saving skeleton-only visualization:", out_file)
        v.save_video(out_file, v.draw())

def find_video(filename=None, root=None):
    root = root or os.getcwd()
    if filename:
        # Try exact path
        if os.path.isabs(filename) and os.path.exists(filename):
            return os.path.abspath(filename)
        rel = os.path.join(root, filename)
        if os.path.exists(rel):
            return os.path.abspath(rel)
    # If no filename or not found, search for mp4/mov/avi
    for ext in ('.mp4', '.mov', '.avi', '.mkv'):
        matches = glob.glob(os.path.join(root, '**', f'*{ext}'), recursive=True)
        if matches:
            return os.path.abspath(matches[0])
    return None

if __name__ == "__main__":
    ROOT = os.getcwd()
    print("Notebook root:", ROOT)

    
    video_filename = "class_2.avi"   # put your video filename here

    video_path = find_video(video_filename, ROOT)
    if not video_path:
        print(f" Video not found. Put '{video_filename}' in {ROOT} or update video_filename.")
        sys.exit(1)

    print(" Using video:", video_path)

    # Make output folder next to video
    out_dir = os.path.join(os.path.dirname(video_path), "output")
    os.makedirs(out_dir, exist_ok=True)
    output_path = os.path.join(out_dir, os.path.splitext(os.path.basename(video_path))[0] + ".mediapipe.pose")

    lib = 'mediapipe'
    visualize = True
    reduce = True

    pose_estimate(video_path, output_path, reduce=reduce, lib=lib)
    if visualize:
        pose_visualize(video_path, output_path, overlay=True)
        pose_visualize(video_path, output_path, overlay=False)
